In [1]:
from stereo3d import stereo_read_from_pickle as std_read
from parsivel import pars_read_from_pickle as pars_read
from pandas import DataFrame
from multifractal_analysis.spectral_analysis import get_beta_and_h
from multifractal_analysis.double_trace_moment import get_um_params_dtm
from multifractal_analysis.general import get_best_slice, fluctuations
from multifractal_analysis.general import split_field
from matplotlib import pyplot as plt
import numpy as np 
from pathlib import Path
pars_folder = Path("/home/marcio/stage_project/data/saved_events/pars_events/")
stereo_folder = Path("/home/marcio/stage_project/data/saved_events/stereo_events/")

Read all the saved events from parsivel and stereo 3d

In [2]:
# Read the main parsivel series
parsivel_events = [pars_read(file_path) for file_path in pars_folder.iterdir()]
stereo_events = [std_read(file_path) for file_path in stereo_folder.iterdir()]

Extract the rain rate from each series

In [3]:
data = [(stereo_event.rain_rate(30), pars_event.rain_rate)for stereo_event, pars_event in zip(stereo_events, parsivel_events)]
#data = [(get_best_slice(str_rate, 2**6), get_best_slice(pars_rate, 2**6)) for str_rate, pars_rate in data]
data = [(split_field(str_rate, 2**6, 0.35), split_field(pars_rate, 2**6)) for str_rate, pars_rate in data]
data = [tup for tup in data if not (tup[0] is None or tup[1] is None)]
data = [(str_rate/np.mean(str_rate), pars_rate/np.mean(pars_rate)) for str_rate, pars_rate in data]
data = [(str_rate, pars_rate) for str_rate, pars_rate in data if not (any(np.isnan(pars_rate)) or any(np.isnan(str_rate)))]
len(data)

0

Spectral Analysis

In [ ]:
results = []
problems = []
for str_rate, pars_rate in data:
    try:
        beta_str, h_str= get_beta_and_h(str_rate)
        beta_pars, h_pars = get_beta_and_h(pars_rate)
        alpha_str, c1_str = get_um_params_dtm(str_rate, 1.5)
        alpha_pars, c1_pars = get_um_params_dtm(pars_rate, 1.5)
        results.append((beta_str, h_str, alpha_str, c1_str, beta_pars, h_pars, alpha_pars, c1_pars))
    except:
        problems.append((str_rate, pars_rate))
        pass

results = DataFrame(data= results, columns=["beta_str", "h_str", "alpha_str", "c1_str", "beta_pars", "h_pars", "alpha_pars", "c1_pars"])
results

In [ ]:
for str_rate, pars_rate in problems:
    plt.plot(str_rate)
    plt.plot(pars_rate)
    break

Calculate the fluctuations for each field

In [ ]:
fluc_data = [(fluctuations(str_rate), fluctuations(pars_rate))for str_rate, pars_rate in data]

Spectral Analysis

In [ ]:
results = []
problems = []
for str_rate, pars_rate in fluc_data:
    try:
        beta_str, h_str= get_beta_and_h(str_rate)
        beta_pars, h_pars = get_beta_and_h(pars_rate)
        alpha_str, c1_str = get_um_params_dtm(str_rate, 1.5)
        alpha_pars, c1_pars = get_um_params_dtm(pars_rate, 1.5)
        results.append((beta_str, h_str, alpha_str, c1_str, beta_pars, h_pars, alpha_pars, c1_pars))
    except:
        problems.append((str_rate, pars_rate))
        pass

results = DataFrame(data= results, columns=["beta_str", "h_str", "alpha_str", "c1_str", "beta_pars", "h_pars", "alpha_pars", "c1_pars"])
results